### Dash
- Dash is a Python framework provided by plotly for building web applications. Written on top of Flask, Plotly.js, and React.js, Dash is ideal for building data visualization apps with highly custom user interfaces in pure Python. It's particularly suited for anyone who works with data in Python.

#### Installation
- **Dash:**
    ```highlighter
    conda install -c conda-forge dash

    ```
    
- **Jupyter Dash:**
    ```highlighter
    conda install -c plotly jupyter-dash

    ```

In [1]:
# Import Necessary libraries
import pandas as pd
import warnings
warnings.filterwarnings("ignore")

# Dash and it's components
from dash import Dash, html, dcc,Input,Output
from jupyter_dash import JupyterDash
import dash_bootstrap_components as dbc 
import plotly.express as px

## Get Data For Analysis

In [2]:
import sys
from pathlib import Path

# in jupyter (lab / notebook), based on notebook path
module_path = str(Path.cwd().parents[0] / "src")

if module_path not in sys.path:
    sys.path.append(module_path)

from scripts.EV_Charging_Analysis import get_charger_type_count_by_country,_clean_and_filter_charging_data,_get_ratio_of_ev_per_cv,_get_charging_ratio_of_countries_by_year

#### Figures Functions

In [3]:
from typing import List,Dict
def _get_charger_by_country(df_:pd.DataFrame, charger_type:str):
    barchart = px.bar(df_, x='year', y='value',color="region_class",
             title=f"{charger_type.title()} publicly available chargers, {'2014' if charger_type=='fast' else '2010'}-2021",
             labels=dict(year="", value=f"{charger_type.title()} Charger Public Stock (Thousands)", region_class="Countries"))
    
#     piechart = px.pie(df_, names="region_class", values="value", color="year")
    
    return barchart

def _get_ratio_ev_vs_cp(ev_per_cp:pd.DataFrame):
    
    fig = px.bar(
            ev_per_cp,
            x= ev_per_cp["ev_per_charging_points"],
            y=ev_per_cp.index.to_list(),
            orientation='h',
            title="Charging points per EV in selected countries, 2021",
            labels=dict(y="Countries", ev_per_charging_points="ELectric Car Per Charging Point"),
            width=1000,
            height=700,
            template="plotly_dark",
#             color_continuous_scale='Bluered_r',
#             color_discrete_map={"China":"green"}
            )
    return fig


def ratio_of_charging_points(df_:pd.DataFrame):
    import plotly.graph_objects as go

    # Create traces
    fig = go.Figure()
    for country in df_.columns:
        fig.add_trace(go.Scatter(x=df_.index, y=df_.loc[:,country],
                            mode='lines+markers',
                            name=country))
    return fig


##### To get started with Dash in Jupyter notebook 
- https://dash.plotly.com/workspaces/using-dash-in-jupyter-and-workspaces

In [4]:
# initialize the app
# app = Dash(__name__)
app = JupyterDash(external_stylesheets=[dbc.themes.SOLAR])

# Build App
# the style arguments for the sidebar. We use position:fixed and a fixed width
SIDEBAR_STYLE = {
    "position": "fixed",
    "top": 0,
    "left": 0,
    "bottom": 0,
    "width": "16rem",
    "padding": "2rem 1rem",
    "background-color": "#f8f9fa",
}

# the styles for the main content position it to the right of the sidebar and
# add some padding.
CONTENT_STYLE = {
    "margin-left": "18rem",
    "margin-right": "2rem",
    "padding": "2rem 1rem",
}

sidebar = html.Div(
    [
        html.H2("Electrical Vehicles Analysis", className="display-4"),
        html.Hr(),
        html.P(
            "Different Components Of Electrical Vehicles For Analysis", className="lead"
        ),
        dbc.Nav(
            [
                dbc.NavLink("Charging Infrastructure", href="/", active="exact"),
#                 dbc.NavLink("EV vs CP Ratio", href="/ev_cp_ratio", active="exact"),
#                 dbc.NavLink("Page 2", href="/page-2", active="exact"),
            ],
            vertical=True,
            pills=True,
        ),
    ],
    style=SIDEBAR_STYLE,
)

content = html.Div(
    children=[
        dbc.Row(dbc.Col(html.H1("Charging Infrastructure Of Electrical Vehicles",style={"textAlign":"center","color":"white","position":"-webkit-sticky"})),align="center"),
        html.Hr(),
        dbc.Row(
            [
                # Country Dropdown
                dbc.Col(html.Div(dcc.Dropdown(options=["China","India","USA","Europe"],
                                                      value="China",
                                                      multi=True,
                                                      id="country_selection"))),
                # Charging Type
#                 dbc.Col(html.Div(dcc.Dropdown(options=["fast","slow"],
#                                                       value="fast",
#                                                       multi=False,
#                                                       id="charging_type"))),
            ],id="dash_component",style={"padding": "2rem 1rem"}),
        
        dbc.Row(
            [
                dbc.Col(html.Div(dcc.Graph(id="bar_chart1")),width=6),
                dbc.Col(html.Div(dcc.Graph(id="bar_chart11")),width=6),
            ],id="dash_graphs",style={"padding": "2rem 1rem"}),
        
        dbc.Row(
            [
                dbc.Col(html.Div(dcc.Graph(id="ratio_bar_chart"))),
            ],id="ratio_chart",style={"padding": "2rem 1rem"}),
        
         dbc.Row(
            [
                dbc.Col(html.Div(dcc.Graph(id="line_chart"))),
            ],id="line_chart",style={"padding": "2rem 1rem"}),
    ],          
    style=CONTENT_STYLE,id="page-content")

app.layout = html.Div([dcc.Location(id="url"), sidebar, content])



@app.callback([Output(component_id="bar_chart1", component_property="figure"),
                Output(component_id="bar_chart11", component_property="figure"),
                Output(component_id="ratio_bar_chart", component_property="figure"),
                Output(component_id="line_chart", component_property="figure")],
                [Input(component_id="url", component_property="pathname"),
                 Input(component_id="country_selection",component_property="value")
#                  Input(component_id="charging_type",component_property="value")
                ])
def render_page_content(pathname,countries):
    df = _clean_and_filter_charging_data()
    if pathname == "/":
        print("countries================",countries if isinstance(countries,list) else [countries])
        fast_data = get_charger_type_count_by_country(df,countries if isinstance(countries,list) else [countries],"fast")
        slow_data = get_charger_type_count_by_country(df,countries if isinstance(countries,list) else [countries],"slow")
        
        # Read Data
        ev_data = pd.read_csv('../Data/IEA-EV-data.csv')
        data = _get_ratio_of_ev_per_cv(ev_data)
        final_data =  _get_charging_ratio_of_countries_by_year(ev_data)
        return _get_charger_by_country(fast_data,"fast"),_get_charger_by_country(slow_data,"slow"),_get_ratio_ev_vs_cp(data),ratio_of_charging_points(final_data)
    
#     elif pathname == "/ev_cp_ratio":
#         pass
    
#     elif pathname == "/page-2":
#         pass
    # If the user tries to reach a different page, return a 404 message
    return html.Div(
        [
            html.H1("404: Not found", className="text-danger"),
            html.Hr(),
            html.P(f"The pathname {pathname} was not recognised..."),
        ],
        className="p-3 bg-light rounded-3",
    )
# start App
if __name__ == '__main__':
    app.run_server(mode="external")

Dash app running on http://127.0.0.1:8050/
countries================ ['China']
countries================ ['China', 'USA']
countries================ ['China', 'USA', 'Europe']
countries================ ['China', 'USA', 'Europe', 'India']
